# PreProcess

In [26]:
#Read DataSet and Stop words

with open ("hafez.txt", "r") as dataset:
    data = dataset.read().splitlines()

with open ("fa_stop_words.txt", "r") as fa_stop_words:
    stop_words = fa_stop_words.read().splitlines()    

In [2]:
print("data:", "\n1: ", data[0], "\n2: ", data[1], "\n3: ", data[2])
print("\nstop_words:", "\n1: ", stop_words[18], "\n2: ", stop_words[19], "\n3: ", stop_words[20])

data: 
1:  ﻿ 
2:  الا يا ايها الساقي ادر كاسا و ناولها 
3:  كه عشق آسان نمود اول ولي افتاد مشكل‌ها

stop_words: 
1:  ؟ 
2:  آباد 
3:  آخ




---



In [ ]:
!pip install hazm

from hazm import *
import numpy as np

In [33]:
normalizer = Normalizer()
normalized_data = []
tokenized_data = []
clear_data = []
clear_sentences = []
words = []
clear_words = []

#### Normalize
for line in data:
  if line and line != '\ufeff':
    normalized_data.append(normalizer.normalize(line))


#### Tokenize
for line in normalized_data:
  tokenized_data.append(word_tokenize(line))


#### Remove stop words
for line in tokenized_data:
  clear_line = []
  for word in line:
    if word not in stop_words:
      clear_line.append(word)
  clear_data.append(clear_line)


#### Create Bag of words
for i in range(len(clear_data)):
  words.extend(clear_data[i])


#### Convert words to sentences
for line in clear_data:
  clear_sentences.append(" ".join(line))
clear_sentences = [item for item in clear_sentences if item]


# Remove duplicate words
for i in words: 
  if i not in clear_words: 
    clear_words.append(i) 


print("Normalized: ", normalized_data[0])
print("Tokenized: ", tokenized_data[0])
print("Cleared: ", clear_data[0])
print("Words: ", words[0], "size: ", len(words))
print("Non Duplicate Words: ", clear_words[0], "size: ", len(clear_words))
print("Sentence: ", clear_sentences[0])

Normalized:  الا یا ایها الساقی ادر کاسا و ناولها
Tokenized:  ['الا', 'یا', 'ایها', 'الساقی', 'ادر', 'کاسا', 'و', 'ناولها']
Cleared:  ['ایها', 'الساقی', 'ادر', 'کاسا', 'ناولها']
Words:  ایها size:  34482
Non Duplicate Words:  ایها size:  8168
Sentence:  ایها الساقی ادر کاسا ناولها


# Process